In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
from monty.serialization import loadfn, dumpfn

In [75]:
from ase.filters import FrechetCellFilter
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.optimize import LBFGS
from pymatgen.io.ase import AseAtomsAdaptor

predictor = pretrained_mlip.get_predict_unit("uma-m-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omat") # use "omat" - inorganic materials

checkpoints/uma-m-1p1.pt:   0%|          | 0.00/11.2G [00:00<?, ?B/s]

In [76]:
def relax_structure(structure): # takes in pymatgen structure
    atoms = structure.to_ase_atoms()
    atoms.calc = calc

    opt = LBFGS(atoms) # Delete this FrechetCellFilter when doing the defect structures
    opt.run(0.01, 1000)

    structure1 = AseAtomsAdaptor.get_structure(atoms)
    energy = atoms.get_potential_energy()

    return structure1, energy 

In [19]:
from doped.chemical_potentials import CompetingPhases
from pymatgen.entries.computed_entries import ComputedEntry
from doped.chemical_potentials import CompetingPhasesAnalyzer

In [77]:
calculated_entries = {}

def calc_competing_phase(comp): # Pass in formula of species
    cp = CompetingPhases(composition=comp, extrinsic='Cr', energy_above_hull=0.05)
    entries = []
    for entry in cp.entries:
        key =  entry.data['material_id'] + entry.structure.composition.reduced_formula
        
        if key in calculated_entries:
            new_entry = calculated_entries[key]
        else:
            relaxed_structure, energy = relax_structure(entry.structure)
            new_entry = ComputedEntry(
                relaxed_structure.composition,
                energy, 
                data = {'material_id':entry.data['material_id']}
            )
            calculated_entries[key] = new_entry 
        entries.append(new_entry)
        
    return cp, entries # maybe dont have to return cp

In [78]:
def calc_competing_phase_analysis(comp, entries):
    
    cpa = CompetingPhasesAnalyzer(composition=comp, entries=entries)
    
    return cpa.chempots

In [48]:
relaxed_defect = loadfn('../data/doped_defect_energies.json')

In [79]:
for key, val in relaxed_defect.items():
    formula = val['formula']
    cp, entries = calc_competing_phase(formula) # do you have to return cp?
    chem_pot = calc_competing_phase_analysis(formula, entries)
    if 'competing phases' not in relaxed_defect[key]:
        relaxed_defect[key]['competing phases'] = {}
    relaxed_defect[key]['competing phases']['chemical potentials'] = chem_pot
    relaxed_defect[key]['competing phases']['entries'] = entries

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: AlPO4_Pbca_EaH_0.012!

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: AlPO4_Pbca_EaH_0.012!



       Step     Time          Energy          fmax
LBFGS:    0 11:41:23     -381.048998        0.033537
LBFGS:    1 11:41:23     -381.049326        0.032588
LBFGS:    2 11:41:24     -381.050169        0.051802
LBFGS:    3 11:41:24     -381.050978        0.056065
LBFGS:    4 11:41:25     -381.051417        0.032707
LBFGS:    5 11:41:25     -381.051611        0.025782
LBFGS:    6 11:41:26     -381.051768        0.018328
LBFGS:    7 11:41:26     -381.051951        0.024989
LBFGS:    8 11:41:27     -381.052126        0.024364
LBFGS:    9 11:41:27     -381.052260        0.021435
LBFGS:   10 11:41:28     -381.052390        0.017935
LBFGS:   11 11:41:28     -381.052504        0.019482
LBFGS:   12 11:41:29     -381.052637        0.015394
LBFGS:   13 11:41:29     -381.052733        0.012416
LBFGS:   14 11:41:30     -381.052798        0.010108
LBFGS:   15 11:41:30     -381.052843        0.009577
       Step     Time          Energy          fmax
LBFGS:    0 11:41:30      -19.058099        0.0000

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: P2O5_NA_EaH_0!



       Step     Time          Energy          fmax
LBFGS:    0 11:59:52     -106.196604        0.053932
LBFGS:    1 11:59:53     -106.196902        0.043495
LBFGS:    2 11:59:53     -106.197472        0.006747
       Step     Time          Energy          fmax
LBFGS:    0 11:59:54       -5.716220        0.008224
       Step     Time          Energy          fmax
LBFGS:    0 11:59:54      -16.060173        0.041232
LBFGS:    1 11:59:54      -16.060231        0.038140
LBFGS:    2 11:59:55      -16.060626        0.009198
       Step     Time          Energy          fmax
LBFGS:    0 11:59:55      -14.110009        0.027185
LBFGS:    1 11:59:55      -14.110041        0.025390
LBFGS:    2 11:59:56      -14.110279        0.002306
       Step     Time          Energy          fmax
LBFGS:    0 11:59:56      -12.002485        0.000001
       Step     Time          Energy          fmax
LBFGS:    0 11:59:57      -13.507328        0.000000
       Step     Time          Energy          fmax
LBFGS: 

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: CrCl3_NA_EaH_0.002!



...

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: Mn2O3_NA_EaH_0!



       Step     Time          Energy          fmax
LBFGS:    0 13:50:58     -494.505500        0.869474
LBFGS:    1 13:50:59     -494.817710        0.297952
LBFGS:    2 13:50:59     -494.857898        0.109246
LBFGS:    3 13:51:00     -494.863037        0.102389
LBFGS:    4 13:51:00     -494.886376        0.073577
LBFGS:    5 13:51:01     -494.887995        0.064535
LBFGS:    6 13:51:02     -494.890829        0.066221
LBFGS:    7 13:51:02     -494.894392        0.082398
LBFGS:    8 13:51:03     -494.899077        0.089627
LBFGS:    9 13:51:03     -494.902776        0.077646
LBFGS:   10 13:51:04     -494.905638        0.080117
LBFGS:   11 13:51:05     -494.908146        0.058344
LBFGS:   12 13:51:05     -494.910477        0.050618
LBFGS:   13 13:51:06     -494.911975        0.038017
LBFGS:   14 13:51:06     -494.912858        0.037508
LBFGS:   15 13:51:07     -494.913746        0.044276
LBFGS:   16 13:51:07     -494.914758        0.037922
LBFGS:   17 13:51:08     -494.915578        0.02

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: Al2O3_NA_EaH_0.0319!



       Step     Time          Energy          fmax
LBFGS:    0 14:07:32     -206.983147        0.087907
LBFGS:    1 14:07:33     -206.983773        0.049532
LBFGS:    2 14:07:33     -206.984112        0.008892
       Step     Time          Energy          fmax
LBFGS:    0 14:07:34       -7.496698        0.000007
       Step     Time          Energy          fmax
LBFGS:    0 14:07:34      -28.423045        0.006741
       Step     Time          Energy          fmax
LBFGS:    0 14:07:36    -1012.390308        0.069045
LBFGS:    1 14:07:38    -1012.391559        0.038042
LBFGS:    2 14:07:39    -1012.392657        0.016007
LBFGS:    3 14:07:41    -1012.392871        0.016077
LBFGS:    4 14:07:43    -1012.393268        0.014147
LBFGS:    5 14:07:44    -1012.393283        0.011526
LBFGS:    6 14:07:46    -1012.393466        0.011388
LBFGS:    7 14:07:48    -1012.393527        0.009156
       Step     Time          Energy          fmax
LBFGS:    0 14:07:48      -14.200244        0.000000
   

/home/atshaam/miniconda3/envs/project_env/lib/python3.11/site-packages/doped/chemical_potentials.py:765: UserWarning:

Duplicate entry names found for generated competing phases: Al2O3_NA_EaH_0.0319!



In [80]:
dumpfn(relaxed_defect, '../data/competing_phase_calcs2.json')

In [3]:
comp_phases = loadfn('../data/competing_phase_calcs2.json')

In [8]:
doped_defect = loadfn('../data/doped_defect_energies2.json')

In [9]:
for key, val in comp_phases.items():
    comp_phases[key]['minimum energy site']['energy'] = doped_defect[key]['minimum energy site']['energy']
    comp_phases[key]['minimum energy site']['information'] = doped_defect[key]['minimum energy site']['information']
    comp_phases[key]['supercell']['energy'] = doped_defect[key]['supercell']['energy']

In [10]:
dumpfn(comp_phases, '../data/final_competing_phase_calcs.json')